In [1]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [2]:
# read in with pandas
winter1 = pd.read_csv ('ddh20-21data.txt')
winter2 = pd.read_csv ('ddh21-22data.txt')
winter3 = pd.read_csv ('ddh22-23data.txt')

In [3]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [4]:
# mask out days without precipitation
codes = ['GS','IC','PL','SG','SN']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [5]:
precip_days2

,station,datetime,precip(mm),wxcodes
0,DDH,2021-12-01 00:54:00,T,-SN BR
5,DDH,2021-12-01 05:54:00,T,-SN BR
6,DDH,2021-12-01 06:08:00,T,-SN
7,DDH,2021-12-01 06:54:00,T,-SN BR
8,DDH,2021-12-01 07:02:00,T,-SN BR
...,...,...,...,...
2929,DDH,2022-02-26 02:49:00,T,-SN BR
2930,DDH,2022-02-26 02:54:00,T,-SN BR
2976,DDH,2022-02-27 22:28:00,T,-SN
2977,DDH,2022-02-27 22:36:00,T,SN


In [6]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_25180\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_25180\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_25180\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [7]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_25180\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_25180\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_25180\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [8]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 54]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 54]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 54]

In [9]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [10]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [11]:
ddh = LocationInfo(
    'Bennington', 'Vermont', 
    latitude=42.8783,
    longitude=-73.1971, 
    timezone='America/New_York'
)

ddh

LocationInfo(name='Bennington', region='Vermont', timezone='America/New_York', latitude=42.8783, longitude=-73.1971)

In [12]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(ddh.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [13]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(ddh.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [14]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(ddh.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [15]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 07:04:44  2020-12-01 16:19:08  2020-12-01
1   2020-12-02 07:05:48  2020-12-02 16:18:52  2020-12-02
2   2020-12-03 07:06:50  2020-12-03 16:18:38  2020-12-03
3   2020-12-04 07:07:51  2020-12-04 16:18:27  2020-12-04
4   2020-12-05 07:08:51  2020-12-05 16:18:18  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:36:27  2021-02-24 17:36:01  2021-02-24
86  2021-02-25 06:34:52  2021-02-25 17:37:17  2021-02-25
87  2021-02-26 06:33:16  2021-02-26 17:38:33  2021-02-26
88  2021-02-27 06:31:40  2021-02-27 17:39:48  2021-02-27
89  2021-02-28 06:30:03  2021-02-28 17:41:03  2021-02-28

[90 rows x 3 columns]


In [16]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 07:04:29  2021-12-01 16:19:12  2021-12-01
1   2021-12-02 07:05:32  2021-12-02 16:18:55  2021-12-02
2   2021-12-03 07:06:35  2021-12-03 16:18:41  2021-12-03
3   2021-12-04 07:07:37  2021-12-04 16:18:29  2021-12-04
4   2021-12-05 07:08:37  2021-12-05 16:18:19  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:36:50  2022-02-24 17:35:43  2022-02-24
86  2022-02-25 06:35:15  2022-02-25 17:36:59  2022-02-25
87  2022-02-26 06:33:39  2022-02-26 17:38:14  2022-02-26
88  2022-02-27 06:32:03  2022-02-27 17:39:30  2022-02-27
89  2022-02-28 06:30:26  2022-02-28 17:40:45  2022-02-28

[90 rows x 3 columns]


In [17]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 07:04:13  2022-12-01 16:19:16  2022-12-01
1   2022-12-02 07:05:17  2022-12-02 16:18:59  2022-12-02
2   2022-12-03 07:06:20  2022-12-03 16:18:44  2022-12-03
3   2022-12-04 07:07:22  2022-12-04 16:18:31  2022-12-04
4   2022-12-05 07:08:22  2022-12-05 16:18:21  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:37:12  2023-02-24 17:35:24  2023-02-24
86  2023-02-25 06:35:38  2023-02-25 17:36:40  2023-02-25
87  2023-02-26 06:34:03  2023-02-26 17:37:56  2023-02-26
88  2023-02-27 06:32:26  2023-02-27 17:39:11  2023-02-27
89  2023-02-28 06:30:50  2023-02-28 17:40:26  2023-02-28

[90 rows x 3 columns]


In [18]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

C:\Users\alyha\AppData\Local\Temp\ipykernel_25180\762931898.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['date'] = precip_days2['datetime'].dt.date


In [19]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,DDH,2020-12-02 18:54:00,0.0001,-SN,2020-12-02,2020-12-02 07:05:48,2020-12-02 16:18:52
1,DDH,2020-12-02 19:54:00,0.0001,-SN,2020-12-02,2020-12-02 07:05:48,2020-12-02 16:18:52
2,DDH,2020-12-02 20:54:00,0.0001,-SN BR,2020-12-02,2020-12-02 07:05:48,2020-12-02 16:18:52
3,DDH,2020-12-05 17:54:00,0.5100,-SN,2020-12-05,2020-12-05 07:08:51,2020-12-05 16:18:18
4,DDH,2020-12-05 18:54:00,0.2500,-SN BR,2020-12-05,2020-12-05 07:08:51,2020-12-05 16:18:18
...,...,...,...,...,...,...,...
244,DDH,2021-02-23 22:54:00,0.2500,-SN BR,2021-02-23,2021-02-23 06:38:01,2021-02-23 17:34:45
245,DDH,2021-02-27 08:54:00,0.0001,-SN,2021-02-27,2021-02-27 06:31:40,2021-02-27 17:39:48
246,DDH,2021-02-27 09:54:00,0.5100,-SN,2021-02-27,2021-02-27 06:31:40,2021-02-27 17:39:48
247,DDH,2021-02-27 10:54:00,0.5100,-SN BR,2021-02-27,2021-02-27 06:31:40,2021-02-27 17:39:48


In [20]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,DDH,2021-12-01 00:54:00,0.0001,-SN BR,2021-12-01,2021-12-01 07:04:29,2021-12-01 16:19:12
1,DDH,2021-12-01 05:54:00,0.0001,-SN BR,2021-12-01,2021-12-01 07:04:29,2021-12-01 16:19:12
2,DDH,2021-12-01 06:54:00,0.0001,-SN BR,2021-12-01,2021-12-01 07:04:29,2021-12-01 16:19:12
3,DDH,2021-12-04 22:54:00,0.5100,-SN,2021-12-04,2021-12-04 07:07:37,2021-12-04 16:18:29
4,DDH,2021-12-08 12:54:00,0.0001,-SN BR,2021-12-08,2021-12-08 07:11:28,2021-12-08 16:18:05
...,...,...,...,...,...,...,...
130,DDH,2022-02-25 22:54:00,0.0001,-SN,2022-02-25,2022-02-25 06:35:15,2022-02-25 17:36:59
131,DDH,2022-02-25 23:54:00,0.0001,-SN,2022-02-25,2022-02-25 06:35:15,2022-02-25 17:36:59
132,DDH,2022-02-26 00:54:00,0.0001,-SN,2022-02-26,2022-02-26 06:33:39,2022-02-26 17:38:14
133,DDH,2022-02-26 01:54:00,0.0001,-SN,2022-02-26,2022-02-26 06:33:39,2022-02-26 17:38:14


In [21]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,DDH,2022-12-01 20:54:00,0.0001,-SN,2022-12-01,2022-12-01 07:04:13,2022-12-01 16:19:16
1,DDH,2022-12-11 16:54:00,0.0001,-SN,2022-12-11,2022-12-11 07:13:54,2022-12-11 16:18:12
2,DDH,2022-12-11 17:54:00,0.0001,-SN,2022-12-11,2022-12-11 07:13:54,2022-12-11 16:18:12
3,DDH,2022-12-11 18:54:00,0.0001,SN,2022-12-11,2022-12-11 07:13:54,2022-12-11 16:18:12
4,DDH,2022-12-11 19:54:00,0.7600,SN,2022-12-11,2022-12-11 07:13:54,2022-12-11 16:18:12
...,...,...,...,...,...,...,...
123,DDH,2023-02-25 20:54:00,0.0001,-SN,2023-02-25,2023-02-25 06:35:38,2023-02-25 17:36:40
124,DDH,2023-02-25 21:54:00,0.0001,-SN,2023-02-25,2023-02-25 06:35:38,2023-02-25 17:36:40
125,DDH,2023-02-25 22:54:00,0.0001,-SN,2023-02-25,2023-02-25 06:35:38,2023-02-25 17:36:40
126,DDH,2023-02-25 23:54:00,0.0001,-SN,2023-02-25,2023-02-25 06:35:38,2023-02-25 17:36:40


In [22]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
8,DDH,2020-12-08 07:54:00,0.0001,-SN,2020-12-08,2020-12-08 07:11:41,2020-12-08 16:18:06
9,DDH,2020-12-08 12:54:00,0.0001,-SN,2020-12-08,2020-12-08 07:11:41,2020-12-08 16:18:06
10,DDH,2020-12-08 13:54:00,0.0001,-SN,2020-12-08,2020-12-08 07:11:41,2020-12-08 16:18:06
12,DDH,2020-12-09 12:54:00,0.0001,-SN,2020-12-09,2020-12-09 07:12:35,2020-12-09 16:18:07
13,DDH,2020-12-09 13:54:00,0.0001,-SN BR,2020-12-09,2020-12-09 07:12:35,2020-12-09 16:18:07
...,...,...,...,...,...,...,...
232,DDH,2021-02-22 15:54:00,0.0001,-SN,2021-02-22,2021-02-22 06:39:34,2021-02-22 17:33:29
245,DDH,2021-02-27 08:54:00,0.0001,-SN,2021-02-27,2021-02-27 06:31:40,2021-02-27 17:39:48
246,DDH,2021-02-27 09:54:00,0.5100,-SN,2021-02-27,2021-02-27 06:31:40,2021-02-27 17:39:48
247,DDH,2021-02-27 10:54:00,0.5100,-SN BR,2021-02-27,2021-02-27 06:31:40,2021-02-27 17:39:48


In [23]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_25180\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_25180\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_25180\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [24]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
9,DDH,2020-12-08 12:54:00,0.0001,-SN,2020-12-08,2020-12-08 07:11:41,2020-12-08 16:18:06,2020-12-08 08:11:41,2020-12-08 15:18:06
10,DDH,2020-12-08 13:54:00,0.0001,-SN,2020-12-08,2020-12-08 07:11:41,2020-12-08 16:18:06,2020-12-08 08:11:41,2020-12-08 15:18:06
12,DDH,2020-12-09 12:54:00,0.0001,-SN,2020-12-09,2020-12-09 07:12:35,2020-12-09 16:18:07,2020-12-09 08:12:35,2020-12-09 15:18:07
13,DDH,2020-12-09 13:54:00,0.0001,-SN BR,2020-12-09,2020-12-09 07:12:35,2020-12-09 16:18:07,2020-12-09 08:12:35,2020-12-09 15:18:07
14,DDH,2020-12-09 14:54:00,0.2500,-SN,2020-12-09,2020-12-09 07:12:35,2020-12-09 16:18:07,2020-12-09 08:12:35,2020-12-09 15:18:07
...,...,...,...,...,...,...,...,...,...
232,DDH,2021-02-22 15:54:00,0.0001,-SN,2021-02-22,2021-02-22 06:39:34,2021-02-22 17:33:29,2021-02-22 07:39:34,2021-02-22 16:33:29
245,DDH,2021-02-27 08:54:00,0.0001,-SN,2021-02-27,2021-02-27 06:31:40,2021-02-27 17:39:48,2021-02-27 07:31:40,2021-02-27 16:39:48
246,DDH,2021-02-27 09:54:00,0.5100,-SN,2021-02-27,2021-02-27 06:31:40,2021-02-27 17:39:48,2021-02-27 07:31:40,2021-02-27 16:39:48
247,DDH,2021-02-27 10:54:00,0.5100,-SN BR,2021-02-27,2021-02-27 06:31:40,2021-02-27 17:39:48,2021-02-27 07:31:40,2021-02-27 16:39:48


In [25]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [26]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3


In [27]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 0.4549236111111111
2021-2022: 0.3563980769230769
2022-2023: 0.09946666666666666


In [28]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,DDH,2020-12-02 18:54:00,0.0001,-SN,2020-12-02,2020-12-02 07:05:48,2020-12-02 16:18:52
1,DDH,2020-12-02 19:54:00,0.0001,-SN,2020-12-02,2020-12-02 07:05:48,2020-12-02 16:18:52
2,DDH,2020-12-02 20:54:00,0.0001,-SN BR,2020-12-02,2020-12-02 07:05:48,2020-12-02 16:18:52
3,DDH,2020-12-05 17:54:00,0.5100,-SN,2020-12-05,2020-12-05 07:08:51,2020-12-05 16:18:18
4,DDH,2020-12-05 18:54:00,0.2500,-SN BR,2020-12-05,2020-12-05 07:08:51,2020-12-05 16:18:18
...,...,...,...,...,...,...,...
240,DDH,2021-02-23 17:54:00,0.0001,-SN,2021-02-23,2021-02-23 06:38:01,2021-02-23 17:34:45
241,DDH,2021-02-23 18:54:00,0.2500,-SN BR,2021-02-23,2021-02-23 06:38:01,2021-02-23 17:34:45
242,DDH,2021-02-23 19:54:00,0.2500,-SN BR,2021-02-23,2021-02-23 06:38:01,2021-02-23 17:34:45
243,DDH,2021-02-23 20:54:00,0.2500,-SN BR,2021-02-23,2021-02-23 06:38:01,2021-02-23 17:34:45


In [29]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_25180\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_25180\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_25180\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [30]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [31]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [32]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.18595903614457832
2021-2022: 0.183151724137931
2022-2023: 0.28581500000000004
